# Gay Berne potential 

Implempentation of freely diffusing rod-like particles with pair interactions given by the Gay-Berne potential. 

In [ ]:
import numpy as np
import msmrd2
import msmrd2.visualization as msmrdvis
from msmrd2.integrators import overdampedLangevin as odLangevin
from msmrd2.potentials import gayBerne
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Define arbitrarily large list of rod-like particles
numparticles = 100
boxsize = 35
D = 1.0
Drot = 1.0
pyPartlist = [] 
for i in range(numparticles):
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False
        # Comment if statement for box boundary
        #if np.linalg.norm(position) > radius:
        #        overlap = True
        #        continue     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 3.0:
                overlap = True
                continue
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    #orientation = np.array([np.cos(np.pi/4.0), np.sin(np.pi/4), 0.0, 0.0]) #Rotate 90 degrees in x axis
    part = msmrd2.particle(D, Drot, position, orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.000005
seed = 0 # Seed = -1 used random device as seed
bodytype = 'rod'
integrator = odLangevin(dt, seed, bodytype) 
integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
# sphericalBoundary = msmrd2.sphere(radius,'reflective')
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Gay Berne potential
potentialGB = gayBerne(3.0,5.0,5.0,1.0)
integrator.setPairPotential(potentialGB)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 20000
stride = 20
datafile  = open('../../data/vmd/potentialGayBerne.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(2*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        v0 = part.position
        v1 = v0 + 1.7*part.orientvector
        v2 = v0 - 1.7*part.orientvector
        if i%stride == 0:
            datafile.write('type_0' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_0' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%1000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
print("Percentage complete: ", 100, " %")

In [ ]:
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_gayBerne(numparticles = numparticles, outfname = "potentialGayBerne", 
                              tclfname = "../../data/vmd/potentialGayBerne_2vmd.tcl")

To load the movie go to /data/vmd and run in a terminal "vmd -e potentialGayBerne_2vmd.tcl".


## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 100
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Position2"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[numparticles-1].position)))
    integrator.integrate(partlist)